In [1]:
#coding=utf-8

from collections import defaultdict
from datetime import datetime
import math
from operator import itemgetter
import os
import random
import re

import numpy as np
import pandas as pd

%matplotlib inline

import little_mallet_wrapper as lmw
import openpyxl

import copy
import csv

import xlwt
import xlrd
import xlutils.copy as xc

# 1 MALLET Path

In [2]:
path_to_mallet = 'C:/mallet/bin/mallet'  # CHANGE THIS TO YOUR MALLET PATH

# 2 Load dataset

16张sheet，每张1大类


In [3]:
dataset_path = '../huawei_appdescrip_spider/4-大类LDA-应用-华为应用市场app分类与功能描述.xls'  # CHANGE THIS TO YOUR DATASET PATH

df_sheets_dict = pd.read_excel(dataset_path,sheet_name = None,encoding="utf-8")
sheets_name = list(df_sheets_dict.keys())

In [4]:
# 测试读某一sheet
df_one_sheet = df_sheets_dict[sheets_name[11]]
df_one_sheet.sample(11)

,分类,子分类,应用名,分词+去停用词+去低频词,迭代去干扰词,迭代去干扰词2
92,儿童,儿童教育,超级粘土史莱姆制作,提供 各式各样 软泥 做 想不到 发挥 创造性 捏 造型 经典 史莱姆 粘液 模拟 制作 打...,提供 各式各样 软泥 做 发挥 创造性 捏 造型 经典 史莱姆 粘液 模拟 制作 打造 喜欢...,提供 各式各样 软泥 做 发挥 创造性 捏 造型 经典 史莱姆 粘液 模拟 制作 打造 喜欢...
190,儿童,儿童教育,托卡迷你城市城堡,来到 托卡 小镇 迷你 城市 城堡 玩耍 朋友 邀请 家里 参观 游玩 朋友 托卡 小镇 四...,来到 托卡 小镇 迷你 城市 城堡 玩耍 朋友 邀请 家里 参观 游玩 朋友 托卡 小镇 四...,来到 托卡 小镇 迷你 城市 城堡 玩耍 朋友 邀请 家里 参观 游玩 朋友 托卡 小镇 四...
1716,儿童,儿童教育,填色小公主,填色 公主 款 简单 益智 软件 锻炼 孩子 思维 能力 家长 解压 神器 随意 选择 喜欢...,填色 公主 款 简单 益智 软件 锻炼 孩子 思维 能力 家长 解压 神器 随意 选择 喜欢...,填色 公主 款 简单 益智 软件 锻炼 孩子 思维 能力 家长 解压 神器 随意 选择 喜欢...
1637,儿童,儿童教育,讯飞儿童手表,款 讯飞 儿童 手表 硬件 设备 配套 讯飞 儿童 手表 绑定 手表 便捷 设置 手表 状...,款 讯飞 儿童 手表 硬件 设备 配套 讯飞 儿童 手表 绑定 手表 便捷 设置 手表 状...,款 讯飞 儿童 手表 硬件 设备 配套 讯飞 儿童 手表 绑定 手表 便捷 设置 手表 状...
461,儿童,儿童教育,美食烹饪厨房,美食 烹饪 厨房 款 任由 烹饪 天下 美食 餐厅 经营 类 带领 走 进 场 天下 美食 ...,美食 烹饪 厨房 款 任由 烹饪 天下 美食 餐厅 经营 类 带领 走 进 场 天下 美食 ...,美食 烹饪 厨房 款 任由 烹饪 天下 美食 餐厅 经营 类 带领 走 进 场 天下 美食 ...
241,儿童,儿童教育,女生美甲涂色化妆大全,时尚 女孩 有趣 美甲 沙龙 提升 美甲 艺术 设计 水平 享受 舒服 手 部 SPA 洗 ...,时尚 女孩 有趣 美甲 沙龙 提升 美甲 艺术 设计 水平 享受 舒服 手 部 spa 洗 ...,时尚 女孩 有趣 美甲 沙龙 提升 美甲 艺术 设计 水平 享受 舒服 手 部 洗 手上 灰...
1350,儿童,儿童教育,换装少女,装 公主 款 公主 形象 装扮 打扮 益智 衣服 配饰 发型 流行 风格 公主 装扮 发型 ...,装 公主 款 公主 形象 装扮 打扮 益智 衣服 配饰 发型 流行 风格 公主 装扮 发型 ...,装 公主 款 公主 形象 装扮 打扮 益智 衣服 配饰 发型 流行 风格 公主 装扮 发型 ...
1694,儿童,儿童教育,中英文绘本故事,快来 中英文 绘本 故事 每晚 宝宝 讲 两 睡 前 故事 中英文 绘本 故事 款 美式 分...,快来 中英文 绘本 故事 每晚 宝宝 讲 两 睡 前 故事 中英文 绘本 故事 款 美式 分...,快来 中英文 绘本 故事 每晚 宝宝 讲 两 睡 前 故事 中英文 绘本 故事 款 美式 分...
848,儿童,儿童教育,组装超级机器人,电流 抓住 机会 屏幕 机器人 组装 新颖 组装 玩法 简单 眼疾手快 变形 金刚 形象 探...,电流 抓住 机会 屏幕 机器人 组装 新颖 组装 玩法 简单 眼疾手快 变形 金刚 形象 探...,电流 抓住 机会 屏幕 机器人 组装 新颖 组装 玩法 简单 眼疾手快 变形 金刚 形象 探...
1409,儿童,儿童教育,儿童欢乐小火车,迷宫 行走 专家 逻辑 思维 观察力 闯 下班 时间 适合 休息 放松 勇 闯 迷宫 简单 ...,迷宫 行走 专家 逻辑 思维 观察力 闯 下班 时间 适合 休息 放松 勇 闯 迷宫 简单 ...,迷宫 行走 专家 逻辑 思维 观察力 闯 下班 时间 适合 休息 放松 勇 闯 迷宫 简单 ...


In [5]:
# 从表格读取数据到list，待输入LDA
training_data = [t for t in df_one_sheet['迭代去干扰词2'].tolist()]
training_data = [str(d).strip() for d in training_data if str(d)]

len(training_data)

2139

In [35]:
#显示有关训练数据集的基本统计信息
lmw.print_dataset_stats(training_data) 

Number of Documents: 1052
Mean Number of Words per Document: 101.5
Vocabulary Size: 12092


# 3 Train topic model

## 3.1 训练 LDA 主题模型

In [7]:
num_topics = 3  # CHANGE THIS TO YOUR PREFERRED NUMBER OF TOPICS

output_directory_path = 'huawei_LDA_output_原分类测试/儿童-3'  # CHANGE THIS TO YOUR OUTPUT DIRECTORY
# output_directory_path = 'huawei_LDA_output_原分类测试/儿童-3' + str(num_topics) # CHANGE THIS TO YOUR OUTPUT DIRECTORY

path_to_training_data           = output_directory_path + '/training.txt'
path_to_formatted_training_data = output_directory_path + '/mallet.training'
path_to_model                   = output_directory_path + '/mallet.model.' + str(num_topics)
path_to_topic_keys              = output_directory_path + '/mallet.topic_keys.' + str(num_topics)
path_to_topic_distributions     = output_directory_path + '/mallet.topic_distributions.' + str(num_topics)
path_to_word_weights            = output_directory_path + '/mallet.word_weights.' + str(num_topics)
path_to_diagnostics             = output_directory_path + '/mallet.diagnostics.' + str(num_topics) + '.xml'

In [8]:

lmw.import_data(path_to_mallet,
                path_to_training_data, #txt格式数据
                path_to_formatted_training_data, #MALLET 格式数据
                training_data) #原始数据

Importing data...
Complete


In [9]:
lmw.train_topic_model(path_to_mallet,
                      path_to_formatted_training_data,
                      path_to_model,
                      path_to_topic_keys,
                      path_to_topic_distributions,
                      path_to_word_weights,
                      path_to_diagnostics,
                      num_topics)

Training topic model...
Complete


## 3.2 主题词

- 为每个主题加载概率最大单词集：每个主题最多20个词


In [134]:
topic_keys_path = output_directory_path + '/mallet.topic_keys.' + str(num_topics)
topic_keys = [line.split('\t')[2].split() for line in open(topic_keys_path, 'r',encoding="utf-8")] 

for i, t in enumerate(topic_keys):
    print(i, '\t', ' '.join(t[:10]))

0 	 客户端 随时随地 个性化 互联网 音乐人 应有尽有 爱好者 电视台 有限公司 沉浸式
1 	 节拍器 调音器 尤克里里 爱好者 吉他谱 架子鼓 演奏家 五线谱 MIDI 随时随地
2 	 电视剧 爱奇艺 优酷全网 NBA 应有尽有 TVB 真人秀 客户端 长津湖 机器人
3 	 随时随地 高颜值 面对面 KTV 应有尽有 陌生人 排位赛 零距离 演唱会 脱口秀
4 	 收音机 Bose 广场舞 脱口秀 CNC 爱好者 黄梅戏 随时随地 鬼吹灯 广播剧
5 	 播放器 娃娃机 均衡器 音频格式 马戏团 推币机 提取器 DSD AAC 随时随地


- 加载每个topic的最高概率主题词分布

In [135]:
# topic_word_probability_dict = lmw.load_topic_word_distributions(output_directory_path + '/mallet.word_weights.' + str(num_topics))

word_weight_path = output_directory_path + '/mallet.word_weights.' + str(num_topics)

topic_word_weight_dict = defaultdict(lambda: defaultdict(float))
topic_sum_dict = defaultdict(float)
with open(word_weight_path,'r',encoding="utf-8") as f:       
    for _line in f:        
        _topic, _word, _weight = _line.split('\t')
        topic_word_weight_dict[_topic][_word] = float(_weight)
        topic_sum_dict[_topic] += float(_weight)

topic_word_probability_dict = defaultdict(lambda: defaultdict(float))
for _topic, _word_weight_dict in topic_word_weight_dict.items():
    for _word, _weight in _word_weight_dict.items():
        topic_word_probability_dict[int(_topic)][_word] = _weight / topic_sum_dict[_topic]

print("=====================共",len(topic_word_probability_dict),"个topic==================")

for _topic, _word_probability_dict in topic_word_probability_dict.items():
    print('Topic', _topic)
    for _word, _probability in sorted(_word_probability_dict.items(), key=lambda x: x[1], reverse=True)[:5]:
        print(round(_probability, 4), '\t', _word)
    print()

=====================共 6 个topic==================
Topic 0
0.0333 	 客户端
0.0247 	 随时随地
0.0229 	 个性化
0.0201 	 互联网
0.0178 	 音乐人

Topic 1
0.0781 	 节拍器
0.0552 	 调音器
0.0414 	 尤克里里
0.0263 	 爱好者
0.0152 	 吉他谱

Topic 2
0.0321 	 电视剧
0.022 	 爱奇艺
0.0173 	 优酷全网
0.0155 	 NBA
0.0119 	 应有尽有

Topic 3
0.0772 	 随时随地
0.0762 	 高颜值
0.0142 	 面对面
0.0132 	 应有尽有
0.0132 	 KTV

Topic 4
0.0457 	 收音机
0.0204 	 Bose
0.0189 	 广场舞
0.0146 	 脱口秀
0.0117 	 CNC

Topic 5
0.0926 	 播放器
0.0274 	 娃娃机
0.0237 	 均衡器
0.0179 	 音频格式
0.0122 	 马戏团



## 3.3 doc主题分布

### 1) 加载每个doc的主题分布

In [10]:
topic_distributions = lmw.load_topic_distributions(output_directory_path + '/mallet.topic_distributions.' + str(num_topics))

len(topic_distributions), len(topic_distributions[0])

assert(len(topic_distributions) == len(training_data))

### 2)获取每个doc的的主题向量：概率从大到小排列

In [12]:
# num_max_p_topics = 2 #取每个doc最可能的topic个数
num_max_p_topics = num_topics #取每个doc最可能的topic个数

indexs_max_p_all = [] #存所有doc的主题向量，主题向量=[(主题,概率),()]

for i in range(len(training_data)): #遍历所有doc
    p_topics = copy.deepcopy(topic_distributions[i]) #每个doc的概率list
    indexs_max_p = [] #存1个doc最可能的topic下标

    for j in range(num_max_p_topics): #找最大值
        max_p = max(p_topics)
        index_max_p = p_topics.index(max_p)
        # indexs_max_p.append((index_max_p,max_p)) #(主题下标，概率) 
        indexs_max_p.append((index_max_p,round(max_p,4))) #(主题下标，概率)
        # indexs_max_p.append((index_max_p,round(max_p,4)*100)) #(主题下标，概率) 概率单位%
        p_topics[index_max_p] = -1 #最大值改0

    indexs_max_p_all.append(indexs_max_p)


### 3)写入主题向量到csv，去掉概率小于0.05的topic

In [13]:
print("==============写入各app的主题向量==============")

f = open("主题向量.csv",'w',encoding='utf-8',newline='')
save_csv = csv.writer(f)

maxnum = len(indexs_max_p_all)
for i in range(maxnum): 
    topic_vector = copy.deepcopy(indexs_max_p_all[i]) #遍历每个app的主题向量=[(主题,概率),()]

    for j in range(num_topics): #遍历主题向量内每个元组，共num_topics个
        if topic_vector[j][1] <= 0.05: 
            topic_vector = topic_vector[:j]
            # print(topic_vector)
            break
        

    save_csv.writerow([v for v in topic_vector]) #避免csv分隔

f.close()

print("==============写入完成==============")

==============写入各app的主题向量==============
==============写入完成==============


### 4)追加写入主题向量到excel

In [207]:
def write_excel_xls_append(path, id_sheet,value):
    index = len(value)  # 获取需要写入数据的行数

    workbook = xlrd.open_workbook(path)  # 打开工作簿
    sheets = workbook.sheet_names()  # 获取工作簿中的所有表格
    worksheet = workbook.sheet_by_name(sheets[id_sheet])  # 获取工作簿中所有表格中的的第一个表格

    rows_old = worksheet.nrows  # 获取表格中已存在的数据的行数
    cols_old = worksheet.ncols  # 获取表格中已存在的数据的行数

    new_workbook = xc.copy(workbook)  # 将xlrd对象拷贝转化为xlwt对象
    new_worksheet = new_workbook.get_sheet(id_sheet)  # 获取转化后工作簿中的第一个表格

    for i in range(0, index):
        for j in range(0, len(value[i])):
            new_worksheet.write(i, j+cols_old, str(value[i][j]))  # 追加写入数据，注意是从i+rows_old行开始写入
    new_workbook.save(path)  # 保存工作簿
    print("xls格式表格【追加】写入数据成功！")

In [210]:
print("==============写入各app的主题向量==============")

data = []
data.append(["主题","概率"])

maxnum = len(indexs_max_p_all)
for i in range(maxnum): 
    topic_vector = copy.deepcopy(indexs_max_p_all[i]) #遍历每个app的主题向量=[(主题,概率),()]
    temp = []

    for j in range(num_topics): #遍历主题向量内每个元组，共num_topics个
        if topic_vector[j][1] <= 0.05: 
            topic_vector = topic_vector[:j]
            break
        else:
            temp.append(topic_vector[j][0])
            temp.append(topic_vector[j][1])

    data.append(temp)
    
write_excel_xls_append("大类LDA-应用-主题向量.xls",0,data)

print("==============写入完成==============")


==============写入各app的主题向量==============
xls格式表格【追加】写入数据成功！
==============写入完成==============


### 5)属于指定topic的概率最高的doc

In [ ]:
# get_top_docs返回元组(概率，训练的doc)
for p, d in lmw.get_top_docs(training_data, topic_distributions, topic_index=0, n=3):
    print(round(p, 4), d) #概率取小数点后4位
    print()

## 3.4 主题分布散度

- 计算两个topic分布之间的 Jensen-Shannon 散度

In [ ]:
lmw.get_js_divergence_topics(0, 7, topic_word_probability_dict)

# Plot topics by category

如果数据集包含一段时间的分类标签，则创建标签 x 主题的热图

In [ ]:
target_labels = ['John Keats', 'Emily Dickinson', 'William Butler Yeats', 'Christina Rossetti']

lmw.plot_categories_by_topics_heatmap(authors,
                                      topic_distributions,
                                      topic_keys, 
                                      output_directory_path + '/categories_by_topics.pdf',
                                      target_labels=target_labels,
                                      dim=(12,4))

如果数据集包含一些时间的分类标签，则创建一组箱线图，每个主题一个图。

In [ ]:
target_labels = ['John Keats', 'Emily Dickinson', 'William Butler Yeats', 'Christina Rossetti']

for _topic_index in range(0, len(topic_keys)):
    
    lmw.plot_categories_by_topic_boxplots(authors,
                                          topic_distributions,
                                          topic_keys, 
                                          _topic_index,
                                          output_path=output_directory_path + '/boxplot.' + str(_topic_index) + '.pdf',
                                          target_labels=target_labels,
                                          dim=(4,4))

# Plot topics over document time

Splits each training document into N chunks, infers the topics for those chunks (using the pre-trained topic model), and plots mean topic probabilities over document time.

In the example shown, it looks like poems often start with physical descriptions and end with abstract and romantic language.

给定一个数据集，将每个文档分成num_chunks个doc

In [ ]:
divided_documents, document_ids, times = lmw.divide_training_data(training_data,
                                                                  num_chunks=10)

len(divided_documents), len(document_ids), len(times)

In [ ]:
path_to_divided_training_data           = output_directory_path + '/training.split.txt'
path_to_divided_formatted_training_data = output_directory_path + '/mallet.split.training'
path_to_divided_topic_distributions     = output_directory_path + '/mallet.split.topic_distributions.' + str(num_topics)

In [ ]:
lmw.import_data(path_to_mallet,
                path_to_divided_training_data,
                path_to_divided_formatted_training_data,
                divided_documents,
                use_pipe_from=path_to_formatted_training_data)

# **模型使用**
使用已在另一组文档上训练的模型获取一组新文档的主题分布

In [ ]:
lmw.infer_topics(path_to_mallet,
                 path_to_model,
                 path_to_divided_formatted_training_data,
                 path_to_divided_topic_distributions)

在训练主题模型后加载每个文档的主题分布

In [ ]:
topic_distributions = lmw.load_topic_distributions(path_to_divided_topic_distributions)

len(topic_distributions), len(topic_distributions[2])

创建线图，每个主题一个，显示文档段的平均主题概率

In [ ]:
for _topic_index in range(0, len(topic_keys)):
    lmw.plot_topics_over_time(topic_distributions, topic_keys, times, _topic_index)